# Best Practices for SQLite with Python

In this notebook, you'll learn the best practices for using SQLite databases with Python applications. These practices will help you write efficient, secure, and maintainable database code.

In [ ]:
import sqlite3
import os
import time
from contextlib import contextmanager

print('Best practices demonstration starting...')

## 1. Connection Management

**Best Practice**: Use context managers for automatic connection handling.

In [ ]:
# BAD: Manual connection management
def bad_connection_example():
    conn = sqlite3.connect('example.db')
    cursor = conn.cursor()
    try:
        cursor.execute('SELECT * FROM users')
        results = cursor.fetchall()
        conn.commit()  # Might forget this
        return results
    except:
        conn.rollback()  # Might forget this
        raise
    finally:
        conn.close()  # Easy to forget

# GOOD: Context manager approach
@contextmanager
def get_db_connection(db_path='example.db'):
    conn = sqlite3.connect(db_path)
    try:
        yield conn
    finally:
        conn.close()

def good_connection_example():
    with get_db_connection() as conn:
        cursor = conn.cursor()
        with conn:  # Automatic transaction
            cursor.execute('SELECT * FROM users')
            return cursor.fetchall()

print('Connection management examples defined')

## 2. Parameterized Queries

**Best Practice**: Always use parameterized queries to prevent SQL injection.

In [ ]:
# BAD: String concatenation (vulnerable to SQL injection)
def bad_query(username, password):
    query = f"SELECT * FROM users WHERE username = '{username}' AND password = '{password}'"
    # This is dangerous!
    return query

# GOOD: Parameterized queries
def good_query(cursor, username, password):
    cursor.execute('SELECT * FROM users WHERE username = ? AND password = ?', 
                   (username, password))
    return cursor.fetchall()

# EVEN BETTER: Named parameters for complex queries
def complex_query(cursor, min_age, max_age, department):
    cursor.execute('''
        SELECT name, age, dept 
        FROM employees 
        WHERE age BETWEEN :min_age AND :max_age 
        AND dept = :dept
        ORDER BY age
    ''', {
        'min_age': min_age,
        'max_age': max_age,
        'dept': department
    })
    return cursor.fetchall()

print('Query parameterization examples defined')

## 3. Transaction Management

**Best Practice**: Use transactions for related operations and handle rollbacks properly.

In [ ]:
# BAD: No transaction management
def bad_transfer_money(from_id, to_id, amount):
    conn = sqlite3.connect('bank.db')
    cursor = conn.cursor()
    
    cursor.execute('UPDATE accounts SET balance = balance - ? WHERE id = ?', (amount, from_id))
    cursor.execute('UPDATE accounts SET balance = balance + ? WHERE id = ?', (amount, to_id))
    
    conn.commit()  # What if this fails?
    conn.close()

# GOOD: Proper transaction handling
def good_transfer_money(from_id, to_id, amount):
    with get_db_connection('bank.db') as conn:
        with conn:
            cursor = conn.cursor()
            
            # Check balance first
            cursor.execute('SELECT balance FROM accounts WHERE id = ?', (from_id,))
            balance = cursor.fetchone()[0]
            
            if balance < amount:
                raise ValueError('Insufficient funds')
            
            # Perform transfer
            cursor.execute('UPDATE accounts SET balance = balance - ? WHERE id = ?', (amount, from_id))
            cursor.execute('UPDATE accounts SET balance = balance + ? WHERE id = ?', (amount, to_id))
            
            # Transaction automatically committed here

print('Transaction management examples defined')

## 4. Error Handling

**Best Practice**: Handle specific exceptions and provide meaningful error messages.

In [ ]:
# BAD: Generic error handling
def bad_user_creation(username, email):
    try:
        conn = sqlite3.connect('users.db')
        cursor = conn.cursor()
        cursor.execute('INSERT INTO users (username, email) VALUES (?, ?)', (username, email))
        conn.commit()
    except Exception as e:
        print(f'Something went wrong: {e}')
    finally:
        conn.close()

# GOOD: Specific error handling
def good_user_creation(username, email):
    try:
        with get_db_connection('users.db') as conn:
            with conn:
                cursor = conn.cursor()
                cursor.execute('INSERT INTO users (username, email) VALUES (?, ?)', (username, email))
                return cursor.lastrowid
                
    except sqlite3.IntegrityError as e:
        if 'username' in str(e):
            raise ValueError(f'Username "{username}" already exists')
        elif 'email' in str(e):
            raise ValueError(f'Email "{email}" already exists')
        else:
            raise ValueError(f'Database constraint violated: {e}')
            
    except sqlite3.OperationalError as e:
        raise RuntimeError(f'Database operation failed: {e}')
        
    except sqlite3.Error as e:
        raise RuntimeError(f'Database error: {e}')

print('Error handling examples defined')

## 5. Database Schema Management

**Best Practice**: Use migrations and validate schema versions.

In [ ]:
# GOOD: Schema management with version tracking
def initialize_database(db_path):
    """Initialize database with proper schema management"""
    
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        
        # Create schema version table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS schema_version (
                version INTEGER PRIMARY KEY,
                applied_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        
        # Check current version
        cursor.execute('SELECT MAX(version) FROM schema_version')
        current_version = cursor.fetchone()[0] or 0
        
        # Apply migrations
        if current_version < 1:
            cursor.execute('''
                CREATE TABLE users (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    username TEXT UNIQUE NOT NULL,
                    email TEXT UNIQUE NOT NULL,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            ''')
            cursor.execute('INSERT INTO schema_version (version) VALUES (1)')
            
        if current_version < 2:
            cursor.execute('ALTER TABLE users ADD COLUMN last_login TIMESTAMP')
            cursor.execute('INSERT INTO schema_version (version) VALUES (2)')
            
        conn.commit()
        
    return True

# Test schema initialization
initialize_database('managed_schema.db')
print('Schema management example completed')

## 6. Connection Pooling

**Best Practice**: For multi-threaded applications, use connection pooling.

In [ ]:
# For simple applications, single connection is fine
# For multi-threaded apps, consider connection pooling

class SimpleConnectionPool:
    """Simple connection pool for demonstration"""
    
    def __init__(self, db_path, max_connections=5):
        self.db_path = db_path
        self.max_connections = max_connections
        self.connections = []
        
    def get_connection(self):
        if self.connections:
            return self.connections.pop()
        elif len(self.connections) < self.max_connections:
            return sqlite3.connect(self.db_path)
        else:
            raise RuntimeError('Connection pool exhausted')
            
    def return_connection(self, conn):
        if len(self.connections) < self.max_connections:
            self.connections.append(conn)
        else:
            conn.close()
            
    def close_all(self):
        for conn in self.connections:
            conn.close()
        self.connections.clear()

# Usage example
pool = SimpleConnectionPool('pooled.db')

# Get connection from pool
conn = pool.get_connection()
cursor = conn.cursor()
cursor.execute('SELECT sqlite_version()')
print(f'SQLite version: {cursor.fetchone()[0]}')

# Return connection to pool
pool.return_connection(conn)

print('Connection pooling example completed')

## 7. Performance Optimization

**Best Practice**: Use appropriate indexes, optimize queries, and batch operations.

In [ ]:
# GOOD: Proper indexing
def setup_indexes(db_path):
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        
        # Create table with indexes
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS products (
                id INTEGER PRIMARY KEY,
                name TEXT NOT NULL,
                category TEXT NOT NULL,
                price REAL NOT NULL,
                in_stock INTEGER DEFAULT 1
            )
        ''')
        
        # Create indexes for frequently queried columns
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_products_category ON products(category)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_products_price ON products(price)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_products_category_price ON products(category, price)')
        
        conn.commit()

# GOOD: Batch operations
def batch_insert_products(products):
    with get_db_connection('products.db') as conn:
        with conn:
            cursor = conn.cursor()
            cursor.executemany('''
                INSERT INTO products (name, category, price, in_stock) 
                VALUES (?, ?, ?, ?)
            ''', products)

# Setup and test
setup_indexes('products.db')

sample_products = [
    ('Laptop', 'Electronics', 999.99, 1),
    ('Mouse', 'Electronics', 29.99, 1),
    ('Book', 'Education', 19.99, 1)
]

batch_insert_products(sample_products)
print('Performance optimization examples completed')

## 8. Security Considerations

**Best Practice**: Implement proper access controls and data validation.

In [ ]:
# GOOD: Input validation and sanitization
import re

def validate_and_create_user(username, email, password):
    """Create user with proper validation"""
    
    # Validate username
    if not re.match(r'^[a-zA-Z0-9_]{3,20}$', username):
        raise ValueError('Username must be 3-20 characters, letters, numbers, underscore only')
    
    # Validate email
    if not re.match(r'^[^@]+@[^@]+\.[^@]+$', email):
        raise ValueError('Invalid email format')
    
    # Validate password strength
    if len(password) < 8:
        raise ValueError('Password must be at least 8 characters')
    
    # Hash password (in real app, use proper hashing)
    hashed_password = f'hash_{password}'  # Placeholder
    
    # Use parameterized query
    with get_db_connection('secure_users.db') as conn:
        with conn:
            cursor = conn.cursor()
            cursor.execute('''
                INSERT INTO users (username, email, password_hash) 
                VALUES (?, ?, ?)
            ''', (username, email, hashed_password))
            
    return True

# GOOD: Limit query results
def search_users_safe(search_term, limit=50):
    """Safe user search with result limiting"""
    with get_db_connection('secure_users.db') as conn:
        cursor = conn.cursor()
        cursor.execute('''
            SELECT username, email FROM users 
            WHERE username LIKE ? 
            LIMIT ?
        ''', (f'%{search_term}%', limit))
        return cursor.fetchall()

print('Security examples defined')

## 9. Backup and Recovery

**Best Practice**: Implement regular backups and recovery procedures.

In [ ]:
# GOOD: Database backup
def backup_database(source_db, backup_path):
    """Create a backup of the database"""
    
    # SQLite backup using SQL
    with sqlite3.connect(source_db) as source:
        with sqlite3.connect(backup_path) as backup:
            source.backup(backup)
    
    print(f'Backup created: {backup_path}')
    return True

# GOOD: WAL mode for better concurrency
def enable_wal_mode(db_path):
    """Enable WAL mode for better performance and concurrency"""
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        cursor.execute('PRAGMA journal_mode=WAL')
        mode = cursor.fetchone()[0]
        print(f'Journal mode: {mode}')
        
        # Also enable foreign keys
        cursor.execute('PRAGMA foreign_keys=ON')
        conn.commit()

# Test backup
backup_database('products.db', 'products_backup.db')
enable_wal_mode('products.db')

print('Backup and recovery examples completed')

## 10. Monitoring and Logging

**Best Practice**: Implement proper logging and monitoring.

In [ ]:
import logging
import time

# Set up database logging
db_logger = logging.getLogger('database')
db_logger.setLevel(logging.INFO)

# Create console handler
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
db_logger.addHandler(handler)

class DatabaseMonitor:
    """Monitor database operations"""
    
    def __init__(self, db_path):
        self.db_path = db_path
        
    @contextmanager
    def monitored_connection(self):
        start_time = time.time()
        conn = sqlite3.connect(self.db_path)
        try:
            db_logger.info(f'Opened connection to {self.db_path}')
            yield conn
        except Exception as e:
            db_logger.error(f'Database error: {e}')
            raise
        finally:
            conn.close()
            duration = time.time() - start_time
            db_logger.info(f'Closed connection (duration: {duration:.3f}s)')

# Usage
monitor = DatabaseMonitor('monitored.db')

with monitor.monitored_connection() as conn:
    cursor = conn.cursor()
    cursor.execute('SELECT 1')
    result = cursor.fetchone()
    db_logger.info(f'Query result: {result}')

print('Monitoring and logging examples completed')

## Summary of Best Practices

1. **Connection Management**: Use context managers for automatic resource cleanup
2. **Parameterized Queries**: Always use placeholders to prevent SQL injection
3. **Transaction Management**: Use transactions for related operations with proper error handling
4. **Error Handling**: Catch specific exceptions and provide meaningful messages
5. **Schema Management**: Use version-controlled migrations
6. **Connection Pooling**: Use pools for multi-threaded applications
7. **Performance**: Create appropriate indexes and use batch operations
8. **Security**: Validate input, hash passwords, limit query results
9. **Backup & Recovery**: Regular backups and WAL mode
10. **Monitoring**: Log operations and monitor performance

Following these practices will help you create robust, secure, and maintainable SQLite applications in Python.

In [ ]:
# Clean up demonstration files
demo_files = [
    'example.db', 'bank.db', 'users.db', 'managed_schema.db', 
    'pooled.db', 'products.db', 'products_backup.db', 'secure_users.db',
    'monitored.db', 'error_handling_demo.db', 'transactions_demo.db',
    'security_demo.db'
]

for file in demo_files:
    if os.path.exists(file):
        os.remove(file)
        print(f'Removed {file}')

print('\nBest practices demonstration completed and cleaned up!')